# Tarefa 3: Implantar um modelo para inferência sem servidor

## Tarefa 3.1: Configuração do ambiente

Instale os pacotes e as dependências.

In [ ]:
#install-dependencies
import boto3
import sagemaker
import sagemaker_datawrangler
import sys
import time

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")

Salve o modelo do laboratório de treinamento e ajuste no bucket-padrão do Amazon Simple Storage Service (Amazon S3). Configure um modelo usando **create_model** e configure **ModelDataUrl** para referenciar o modelo treinado.

In [ ]:
#set-up-model
# Upload the model to your Amazon S3 bucket
s3_client.upload_file(
    Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz"
)

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

## Tarefa 3.2: Criar um endpoint por meio do modelo retreinado e sintetizado fornecido

O Amazon SageMaker Serverless Inference é uma opção de inferência com propósito definido que ajuda a implantar e dimensionar modelos de machine learning (ML). O Serverless Inference é ideal para cargas de trabalho que apresentam períodos de ociosidade entre os picos de tráfego e que podem tolerar inicialização a frio. Os endpoints sem servidor iniciam recursos de computação automaticamente e aumentam ou reduzem a quantidade dependendo do tráfego. Portanto, você não precisa escolher os tipos de instância nem gerenciar as políticas de scaling. Isso elimina o trabalho pesado de selecionar e gerenciar servidores. O Serverless Inference se integra ao AWS Lambda para oferecer alta disponibilidade, tolerância a falhas integrada e auto scaling.

Há três etapas para criar um endpoint sem servidor usando o SDK Python do Amazon SageMaker. Estas etapas são as mesmas usadas para os endpoints em tempo real, mas apresentam configurações diferentes:
1. Criar um modelo do SageMaker no SageMaker.
2. Criar uma configuração de endpoint para um endpoint HTTPS.
3. Criar um endpoint HTTPS.

Você já criou um modelo. Agora você já pode criar uma configuração de endpoint e um endpoint. 

Primeiro, defina o nome da configuração de endpoint e o tamanho da memória que deseja usar. Depois, chame a API CreateEndpointConfig.

Para criar uma configuração de endpoint, você precisa definir as seguintes opções:
- **VariantName**: o nome da variante de produção (um ou mais modelos em produção).
- **ModelName**: o nome do modelo que você deseja hospedar. Esse é o nome que você especificou quando criou o modelo.
- **ServerlessConfig**: é nessa opção que o endpoint é definido como sem servidor. Configure o valores de **MemorySizeInMB** e **MaxConcurrency**.
    - **MemorySizeInMB**: o tamanho de memória alocado (1024, 2048, 3072, 4096, 5120 ou 6144 MB).
    - **MaxConcurrency**: o número de invocações simultâneas (1 a 200).

In [ ]:
#create-endpoint-configuration 
# Create an endpoint config name. Here you create one based on the date so you can search endpoints based on creation time.
endpoint_config_name = 'income-model-serverless-endpoint-{}'.format(create_date)                              

endpoint_config_response = sm.create_endpoint_config(
   EndpointConfigName=endpoint_config_name,
   ProductionVariants=[
        {
            "ModelName": model_name,
            "VariantName": "variant1", # The name of the production variant
            "ServerlessConfig": {
                "MemorySizeInMB": 2048, # The memory size
                "MaxConcurrency": 20 # Number of concurrent invocations
            }
        } 
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

Em seguida, crie um endpoint. Quando você cria um endpoint sem servidor, o SageMaker provisiona e gerencia os recursos de computação. Depois, você pode fazer solicitações de inferência para o endpoint e receber previsões de modelo em resposta. O SageMaker aumenta e reduz a quantidade de recursos de computação conforme a necessidade para lidar com o tráfego da solicitação e você só para pelo que usa.

Você pode escolher um contêiner fornecido pelo SageMaker ou usar o seu. Um endpoint sem servidor tem o tamanho mínimo de RAM de 1024 MB e máximo de 6144 MB. O Serverless Inference atribui automaticamente os recursos de computação proporcionais à memória selecionada.

Quando o endpoint está em serviço, a função auxiliar imprime o Amazon Resource Name (ARN) do endpoint. A criação do endpoint pode exigir até sete minutos para ser executada.

In [ ]:
#create-endpoint
# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = '{}-name'.format(endpoint_config_name)

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
) 

def wait_for_endpoint_creation_complete(endpoint):
    """Helper function to wait for the completion of creating an endpoint"""
    response = sm.describe_endpoint(EndpointName=endpoint_name)
    status = response.get("EndpointStatus")
    while status == "Creating":
        print("Waiting for Endpoint Creation")
        time.sleep(15)
        response = sm.describe_endpoint(EndpointName=endpoint_name)
        status = response.get("EndpointStatus")

    if status != "InService":
        print(f"Failed to create endpoint, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create endpoint {create_endpoint_response['EndpointArn']}, status: {status}, reason: {failureReason}"
        )
    print(f"Endpoint {create_endpoint_response['EndpointArn']} successfully created.")

wait_for_endpoint_creation_complete(endpoint=create_endpoint_response)


No SageMaker Studio, você pode visualizar os detalhes do endpoint na aba **Endpoints**.

A próxima etapa abre uma nova aba no SageMaker Studio. Para seguir essas orientações, escolha uma das seguintes opções:
- **Opção 1:** visualizar as abas lado a lado. Para criar uma visualização de tela dividida por meio da janela principal do SageMaker Studio, arraste a aba **serverless_inference.ipynb** para a lateral ou selecione (clique com o botão direito) a aba **serverless_inference.ipynb** e escolha **New View for Notebook** (Nova visualização do notebook). Agora, as orientações ficam visíveis enquanto você explora o endpoint.
- **Opção 2:** alternar entre as abas do SageMaker Studio para seguir essas instruções. Ao terminar de explorar o endpoint, retorne ao notebook selecionando a aba **serverless_inference.ipynb**.

1. Selecione o ícone **Página inicial do SageMaker**.
2. Escolha **Deployments** (Implantações).
3. Escolha **Endpoints**.

O SageMaker Studio exibe a aba **Endpoints**.

4. Selecione o endpoint que tem **income-model-serverless-** na coluna **Name** (Nome).

Se o endpoint não aparecer, selecione o ícone de atualização até que o endpoint apareça.

O SageMaker Studio exibe a aba **ENDPOINT DETAILS** (DETALHES DO ENDPOINT).

5. Selecione a aba **AWS settings** (Configurações da AWS).

Se você abriu o endpoint antes que a criação fosse concluída, selecione o ícone de atualização até que o **Endpoint status** (Status do endpoint) seja alterado de *Creating* para *InService*.

O **Endpoint type** (Tipo de endpoint) está listado como **Serverless** (Sem servidor). A seção **Endpoint runtime settings** (Configurações de runtime do endpoint) mostra as configurações que você já escolheu no notebook.

## Tarefa 3.3: Invocar um endpoint para uma inferência sem servidor com registros de clientes

Depois de implantar o modelo usando os serviços de host do SageMaker, você poderá testar o modelo nesse endpoint enviando dados a ele.

Se o endpoint não receber tráfego durante um tempo e de repente passar a receber novas solicitações, poderá demorar um pouco até que o endpoint ative os recursos de computação para processar as solicitações. Isto é chamado de inicialização a frio. Como os endpoints sem servidor provisionam recursos de computação sob demanda, o endpoint poderá passar por inicializações a frio. Uma inicialização a frio também pode ocorrer quando as solicitações simultâneas excedem o uso de solicitação simultânea atual. O tempo de inicialização a frio depende do tamanho do modelo, da duração do download do modelo e do tempo de inicialização do contêiner.

Consulte [Serverless Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints.html) para saber como funciona a inferência sem servidor e a inicialização a frio.

Você recebeu vários registros de cliente adicionais. Confirme se o endpoint está funcionando invocando-o com um conjunto de registros que tenham valor de receita igual a 1 e registros com valor de receita igual a 0. Uma lista de pontuações de previsão para cada registro é gerada. 

In [ ]:
#invoke-endpoint-serverless-records
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

response = sagemaker_runtime.invoke_endpoint(
    ContentType='text/csv',
    EndpointName=endpoint_name, 
    Body=bytes('47,0,4,9,0,3,4,0,1,0,1902,60\n' +
                '53,0,0,0,0,2,4,0,1,0,0,40\n' +
                '44,0,0,0,2,0,1,0,1,14344,0,40\n', 'utf-8')
)

print(response)

print('\nTesting with records that have an income value of 1:')
print('The returned scores are: {}'.format(response['Body'].read().decode('utf-8')))

start_time = time.time()
response = sagemaker_runtime.invoke_endpoint(
    ContentType='text/csv',
    EndpointName=endpoint_name, 
    Body=bytes('19,0,1,1,1,1,2,1,0,0,0,35\n' +
                '56,2,1,1,0,1,0,0,0,0,0,50\n' +
                '61,2,0,0,0,0,0,0,0,0,0,40\n', 'utf-8')
)

print('\nTesting with records that have an income value of 0:')
print('The returned scores are: {}'.format(response['Body'].read().decode('utf-8')))

## Tarefa 3.4: Excluir o endpoint

A limpeza do endpoint pode ser realizada em três etapas. Primeiro, exclua o endpoint. Depois, exclua a configuração do endpoint. Por fim, se você não precisar mais do modelo implantado, exclua-o.

In [ ]:
#delete-resources
# Delete endpoint
sm.delete_endpoint(EndpointName=endpoint_name)

# Delete endpoint configuration
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
                   
# Delete model
sm.delete_model(ModelName=model_name)

### Conclusão

Parabéns! Você usou o SageMaker para criar um endpoint sem servidor usando o SDK Python do SageMaker e para invocar o endpoint com sucesso.

A próxima tarefa do laboratório se concentra na implantação de um modelo para inferência usando a inferência assíncrona.

## Limpeza

Você concluiu este notebook. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de notebook.
- Retorne à sessão do laboratório e continue na **Tarefa 4: Implantar um modelo para inferência assíncrona**.